# Compare One Day of Data

Created by Mitas Ray on 2024-12-09.

Last edited by Mitas Ray on 2024-12-09.

This notebook investigates three anomalies based on a single day of data for two datasets: (1) current data used for automated training, and (2) new data to be used for automated training. The anomalies are: (1) the data in the sets have minor differences, (2) the same model has vastly different predictions on these two datasets, (3) a model that was trained on just two epochs has very similar accuracy to a model that was trained on 100 epochs. The procedure is to 
1. download both datasets and isolate the single day
2. download the archived models
3. compare the accuracy results of the archived models on these datasets

To run the notebook, use Python 3.10 (Python 3.12 does not work), and
- on linux: use `ficc_python/requirements_py310_linux_jupyter.txt`
- on mac: use `ficc_python/requirements_py310_mac_jupyter.txt`

Note: This notebook **requires at least 80 GB of RAM** on a VM. On a MacBook Pro M1 Max with 32 GB of RAM, it can still run because swap memory allows the system to use additional storage space to supplement the required memory.

Change the following files and/or variables to enable credentials and the correct directories:
- `automated_training_auxiliary_functions.py::get_creds(...)` to be the location of the credentials file
- `automated_training_auxiliary_variables.py::WORKING_DIRECTORY` to be the location of the current working directory

In [1]:
# loads the autoreload extension
%load_ext autoreload
# automatically reloads all imported modules when their source code changes
%autoreload 2

In [2]:
import os

import pandas as pd
from tensorflow import keras


# importing from parent directory: https://stackoverflow.com/questions/714063/importing-modules-from-parent-folder
import sys
sys.path.insert(0, '../../')


from automated_training_auxiliary_variables import WORKING_DIRECTORY
from automated_training_auxiliary_functions import STORAGE_CLIENT, create_input, load_model, create_summary_of_results
from ficc.utils.gcp_storage_functions import download_data

INFO: Pandarallel will run on 5 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.
Initialized pandarallel with 5 cores
INFO: Pandarallel will run on 5 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.
Initialized pandarallel with 5 cores
INFO: Pandarallel will run on 5 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.
Initialized pandarallel with 5 cores
In PRODUCTION mode (to change to TESTING mode, set `TESTING` to `True`); all files and models will be saved and NUM_EPOCHS=100
INFO: Pandarallel will run on 5 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.
Initialized pandarallel with 5 cores


In [3]:
AUTOMATED_TRAINING_BUCKET = 'automated_training'

In [4]:
def get_data_for_automated_training_and_isolate_to_single_dates(old_or_new: str, dates: list) -> pd.DataFrame:
    assert old_or_new in ('old', 'new')
    df_list = []
    df_downloaded_from_google_cloud_storage = {}
    for date in dates:
        pickle_file_path = f'{WORKING_DIRECTORY}/files/{old_or_new}_data_{date}.pkl'
        if os.path.exists(pickle_file_path):
            print(f'Loading pickle file from {pickle_file_path}')
            df_list.append(pd.read_pickle(pickle_file_path))
        else:
            print(f'Could not find pickle file in {pickle_file_path}, so creating it now')
            suffix = '' if old_or_new == 'old' else '_v2'
            google_cloud_storage_file_name =  f'processed_data_yield_spread_with_similar_trades{suffix}.pkl'
            if google_cloud_storage_file_name not in df_downloaded_from_google_cloud_storage:
                df = download_data(STORAGE_CLIENT, AUTOMATED_TRAINING_BUCKET, google_cloud_storage_file_name)
                df_downloaded_from_google_cloud_storage[google_cloud_storage_file_name] = df
            else:
                df = df_downloaded_from_google_cloud_storage[google_cloud_storage_file_name]
            
            df = df[df['trade_date'] == date]
            df.to_pickle(pickle_file_path)
            df_list.append(df)
    return df_list

In [ ]:
current_df_on_2024_12_06 = get_data_for_automated_training_and_isolate_to_single_dates('old', ['2024-12-06'])
new_df_on_2024_12_06 = get_data_for_automated_training_and_isolate_to_single_dates('new', ['2024-12-06'])

Could not find pickle file in /Users/mitas/ficc/ficc_python/notebooks/compare_datasets/files/old_data_2024-12-06.pkl, so creating it now


### Anomaly 1: different data in the two datasets
Check which RTRS control numbers are differing.

In [ ]:
print(f'Number of items in the current df: {len(current_df_on_2024_12_06)}')
print(f'Number of items in the new df: {len(new_df_on_2024_12_06)}')

In [ ]:
current_df_on_2024_12_06_rtrs_control_numbers = set(current_df_on_2024_12_06['rtrs_control_number'].tolist())
new_df_on_2024_12_06_rtrs_control_numbers = set(new_df_on_2024_12_06['rtrs_control_number'].tolist())

In [ ]:
print(f'RTRS control numbers in the current df but not in the new df: {current_df_on_2024_12_06_rtrs_control_numbers - new_df_on_2024_12_06_rtrs_control_numbers}')
print(f'RTRS control numbers in the new df but not in the current df: {new_df_on_2024_12_06_rtrs_control_numbers - current_df_on_2024_12_06_rtrs_control_numbers}')

### Anomaly 2: same model has vastly different predictions

In [ ]:
similar_trades_model_2024_12_06, _ = load_model('2024-12-06', 'yield_spread_with_similar_trades')

In [ ]:
create_summary_of_results(similar_trades_model_2024_12_06, current_df_on_2024_12_06, *create_input(current_df_on_2024_12_06), print_results=False)

In [ ]:
create_summary_of_results(similar_trades_model_2024_12_06, new_df_on_2024_12_06, *create_input(current_df_on_2024_12_06), print_results=False)

### Anomaly 3: test model has similar accuracy to production model
The v2 yield spread with similar trades model trained on 2024-12-06 used only 2 epochs.

In [ ]:
similar_trades_model_2024_12_09, _ = load_model('2024-12-09', 'yield_spread_with_similar_trades')
similar_trades_model_v2_2024_12_09 = keras.models.load_model(os.path.join(AUTOMATED_TRAINING_BUCKET, f'similar-trades-v2-model-2024-12-09'))    # create path of the form: <bucket>/<model>